In [4]:
cd ..

/Users/sathvikavasarala/Desktop/codes/r1-1.5b-interp


/Users/sathvikavasarala/miniconda3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'

In [6]:
import json

def read_jsonl_basic(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Skip empty lines
                json_obj = json.loads(line)
                data.append(json_obj)
    return data


In [7]:
data = read_jsonl_basic(file_path)

In [15]:
from dotenv import load_dotenv
import os
import time
import json
import math
import concurrent.futures
from together import Together

# Load environment variables from .env file
load_dotenv()

# Access the API key and set it in the environment
api_key = os.getenv('TOGETHER_API_KEY')
os.environ["TOGETHER_API_KEY"] = api_key

client = Together()

def fetch_response(question, model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"):
    """
    Makes a single API call to fetch a response for the given question using streaming.
    
    Returns:
        str: The final concatenated response.
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": "Please reason step by step, and put your final answer within \\boxed{} " + question
            },
        ],
        max_tokens=None,
        temperature=0.6,
        top_p=0.95,
        top_k=None,
        repetition_penalty=1,
        stop=["<｜end of sentence｜>"],
        stream=True,
    )
    current_response = []
    for token in response:
        if hasattr(token, 'choices'):
            current_response.extend([choice.delta.content for choice in token.choices])
    return "".join(current_response)

def get_responses_and_save(question, num_responses=64, output_file="output.jsonl", model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"):
    """
    Fetches responses concurrently in batches of 90 and writes each response immediately to a JSONL file.
    
    After sending each batch of 90 calls concurrently, the function waits until 90 seconds have passed
    since the start of that batch before starting the next batch.
    
    Args:
        question (str): The question to ask.
        num_responses (int): Total number of responses to generate.
        output_file (str): Path to the JSONL file where responses will be saved.
    """
    batch_size = 90
    total_batches = math.ceil(num_responses / batch_size)
    global_count = 0
    output_file = 'output' + model_name.split('/')[-1] + '.jsonl'

    with open(output_file, "a") as f:
        for batch in range(total_batches):
            start_batch = time.time()
            # Determine the number of requests in this batch
            current_batch_size = min(batch_size, num_responses - batch * batch_size)
            print(f"Starting batch {batch + 1} with {current_batch_size} requests.")

            # Use ThreadPoolExecutor to fire off concurrent API calls
            with concurrent.futures.ThreadPoolExecutor(max_workers=current_batch_size) as executor:
                # Map each future to its batch index
                future_to_index = {
                    executor.submit(fetch_response, question, model_name): i
                    for i in range(current_batch_size)
                }
                for future in concurrent.futures.as_completed(future_to_index):
                    batch_index = future_to_index[future]
                    try:
                        final_response = future.result()
                    except Exception as e:
                        final_response = f"Error: {e}"
                    global_count += 1
                    result = {
                        "question": question,
                        "generated_response": final_response
                    }
                    f.write(json.dumps(result) + "\n")
                    f.flush()
                    print(f"Saved response {global_count}/{num_responses}")

            # Enforce waiting until 60 seconds have passed since the batch started
            elapsed = time.time() - start_batch
            if elapsed < 60:
                wait_time = 60 - elapsed
                print(f"Batch {batch + 1} complete. Waiting {wait_time:.2f} seconds for rate limit.")
                time.sleep(wait_time)
            else:
                print(f"Batch {batch + 1} took {elapsed:.2f} seconds, no wait needed.")


In [ ]:

# Example usage:
num_responses = 64  # Total desired responses

for i, question_data in enumerate(data):
    question = question_data['question']
    get_responses_and_save(question, num_responses, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")

    print(f'{i+1}/{len(data)} done')

Starting batch 1 with 60 requests.
Saved response 1/64
Saved response 2/64
Saved response 3/64
Saved response 4/64
Saved response 5/64
Saved response 6/64
Saved response 7/64
Saved response 8/64
Saved response 9/64
Saved response 10/64
Saved response 11/64
Saved response 12/64
Saved response 13/64
Saved response 14/64
Saved response 15/64
Saved response 16/64
Saved response 17/64
Saved response 18/64
Saved response 19/64
Saved response 20/64
Saved response 21/64
Saved response 22/64
Saved response 23/64
Saved response 24/64
Saved response 25/64
Saved response 26/64
Saved response 27/64
Saved response 28/64
Saved response 29/64
Saved response 30/64
Saved response 31/64
Saved response 32/64
Saved response 33/64
Saved response 34/64
Saved response 35/64
Saved response 36/64
Saved response 37/64
Saved response 38/64
Saved response 39/64
Saved response 40/64
Saved response 41/64
Saved response 42/64
Saved response 43/64
Saved response 44/64
Saved response 45/64
Saved response 46/64
Saved re

In [8]:
import json
import math
import time
import os
import concurrent.futures
from dotenv import load_dotenv
from together import Together

# Load environment variables from .env file and set the API key
load_dotenv()
api_key = os.getenv('TOGETHER_API_KEY')
os.environ["TOGETHER_API_KEY"] = api_key
client = Together()

def fetch_response(question, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", max_retries=3):
    """
    Makes a single API call to fetch a response for the given question using streaming.
    Includes error handling and retries on rate-limit errors.
    
    Returns:
        str: The concatenated response.
    """
    retry_count = 0
    delay = 5  # initial delay in seconds before retrying
    while True:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "user",
                        "content": "Please reason step by step, and put your final answer within \\boxed{} " + question
                    },
                ],
                max_tokens=None,
                temperature=0.6,
                top_p=0.95,
                top_k=None,
                repetition_penalty=1,
                stop=["<｜end of sentence｜>"],
                stream=True,
            )
            current_response = []
            for token in response:
                if hasattr(token, 'choices'):
                    current_response.extend([choice.delta.content for choice in token.choices])
            return "".join(current_response)
        except Exception as e:
            # Check if it's a rate limit error (HTTP 429)
            if "429" in str(e) and retry_count < max_retries:
                retry_count += 1
                print(f"Rate limit hit. Retry {retry_count}/{max_retries} in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
                continue
            # For non-rate limit errors or after max retries, raise the error
            raise e

def get_all_responses(questions, responses_per_question=64, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"):
    """
    Creates tasks for every (question, response) pair and processes them in batches of 90 
    (the model-specific rate limit). Each API call's output is saved to a JSONL file along with
    its question id and response index.
    """
    # Build a list of tasks with metadata
    tasks = []
    for q_idx, q_data in enumerate(questions):
        question_text = q_data['question']
        for r in range(responses_per_question):
            tasks.append({
                "question_id": q_idx,
                "question": question_text,
                "response_index": r
            })
    total_tasks = len(tasks)
    batch_size = 90  # fully use the rate limit of 90 queries per minute
    total_batches = math.ceil(total_tasks / batch_size)
    
    # Create an output file (named using the model's short name)
    output_file = 'output' + model_name.split('/')[-1] + '.jsonl'
    
    with open(output_file, "w") as f:
        with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
            task_idx = 0
            for batch in range(total_batches):
                start_batch = time.time()
                current_batch = tasks[task_idx:task_idx+batch_size]
                future_to_task = {}
                # Submit all tasks in this batch
                for task in current_batch:
                    future = executor.submit(fetch_response, task["question"], model_name)
                    future_to_task[future] = task
                
                # As each future completes, write its result with metadata
                for future in concurrent.futures.as_completed(future_to_task):
                    task_info = future_to_task[future]
                    try:
                        response_text = future.result()
                    except Exception as e:
                        response_text = f"Error: {e}"
                    output_record = {
                        "question_id": task_info["question_id"],
                        "question": task_info["question"],
                        "response_index": task_info["response_index"],
                        "generated_response": response_text
                    }
                    f.write(json.dumps(output_record) + "\n")
                    f.flush()
                    print(f"Saved response for question {task_info['question_id']} (response index {task_info['response_index']})")
                
                task_idx += batch_size
                # Wait until 60 seconds have passed since the start of the batch
                elapsed = time.time() - start_batch
                if elapsed < 60:
                    wait_time = 60 - elapsed
                    print(f"Batch {batch + 1}/{total_batches} complete. Waiting {wait_time:.2f} seconds for rate limit.")
                    time.sleep(wait_time)
                else:
                    print(f"Batch {batch + 1}/{total_batches} took {elapsed:.2f} seconds, no wait needed.")
    print(f"All {total_tasks} responses saved to {output_file}")
    return output_file

def concatenate_responses(input_file, output_file="concatenated_output.jsonl", responses_per_question=64):
    """
    Reads the raw JSONL output file and groups responses by question.
    Every group (question) is assumed to have responses_per_question items.
    The final concatenated data is saved to a new JSONL file.
    """
    with open(input_file, "r") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
    
    # Group responses by question_id
    grouped = {}
    for item in raw_data:
        qid = item["question_id"]
        if qid not in grouped:
            grouped[qid] = {
                "question": item["question"],
                "generated_responses": []
            }
        grouped[qid]["generated_responses"].append((item["response_index"], item["generated_response"]))
    
    # Sort responses within each question by their response_index
    concatenated_data = []
    for qid in sorted(grouped.keys()):
        group = grouped[qid]
        # Sort by response_index to preserve the intended order
        sorted_responses = [resp for idx, resp in sorted(group["generated_responses"], key=lambda x: x[0])]
        # In case there are extra responses, take only the first responses_per_question items
        group["generated_responses"] = sorted_responses[:responses_per_question]
        concatenated_data.append(group)
    
    # Write the concatenated output, one JSON per question
    with open(output_file, "w") as f:
        for item in concatenated_data:
            f.write(json.dumps(item) + "\n")
    print(f"Concatenated output saved to {output_file}")
    print(f"Total questions processed: {len(concatenated_data)}")

if __name__ == "__main__":
    # Read your input questions from the JSONL file
    input_file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'
    with open(input_file_path, "r") as f:
        questions = [json.loads(line) for line in f if line.strip()]
    
    responses_per_question = 64  # the number of responses you actually need per question
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # your model of choice
    
    # Get all responses concurrently (using batches of 90 to fully use the rate limit)
    raw_output_file = get_all_responses(questions, responses_per_question, model_name)
    
    # Now concatenate responses by grouping every 64 responses per question and save to a new file.
    concatenate_responses(raw_output_file, "concatenated_output.jsonl", responses_per_question)


Saved response for question 0 (response index 38)
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Rate limit hit. Retry 1/3 in 5 seconds...
Saved response for question 0 (response index 15)
Rate limit hit. Re

In [10]:
import json
import math
import time
import os
import concurrent.futures
from dotenv import load_dotenv
from together import Together

# Load environment variables from .env file and set the API key
load_dotenv()
api_key = os.getenv('TOGETHER_API_KEY')
os.environ["TOGETHER_API_KEY"] = api_key
client = Together()

def fetch_response(question, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", max_retries=3):
    """
    Makes a single API call to fetch a response for the given question using streaming.
    Includes error handling and retries on rate-limit (429) or server overload (503) errors.
    
    Returns:
        str: The concatenated response.
    """
    retry_count = 0
    delay = 5  # initial delay (seconds) before retrying
    while True:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "user",
                        "content": "Please reason step by step, and put your final answer within \\boxed{} " + question
                    },
                ],
                max_tokens=None,
                temperature=0.6,
                top_p=0.95,
                top_k=None,
                repetition_penalty=1,
                stop=["<｜end of sentence｜>"],
                stream=True,
            )
            current_response = []
            for token in response:
                if hasattr(token, 'choices'):
                    current_response.extend([choice.delta.content for choice in token.choices])
            result = "".join(current_response)
        except Exception as e:
            # If an exception is raised, check for rate limit (429) or server overload (503) errors
            if "429" in str(e) and retry_count < max_retries:
                retry_count += 1
                print(f"Exception hit: {e}. Retry {retry_count}/{max_retries} in {delay} seconds...")
                time.sleep(delay)
                delay *= 2
                continue
            raise e

        # Check if the result itself is an error message (even if no exception was thrown)
        if result.strip().lower().startswith("error") and (("429" in result or "503" in result) and retry_count < max_retries):
            retry_count += 1
            print(f"Received error response: {result}. Retrying {retry_count}/{max_retries} in {delay} seconds...")
            time.sleep(delay)
            delay *= 2
            continue

        return result

def get_existing_raw_data(output_file):
    """
    Reads the raw output file (if it exists) and returns a dictionary mapping each question_id
    to the list of response indices already generated, filtering out any responses that start with "error".
    """
    existing = {}
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    record = json.loads(line)
                    # Skip this record if the generated response starts with "error"
                    response = record.get("generated_response", "")
                    if response.strip().lower().startswith("error"):
                        continue
                    qid = record["question_id"]
                    if qid not in existing:
                        existing[qid] = []
                    existing[qid].append(record["response_index"])
    return existing

def get_all_responses(questions, responses_per_question=64, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"):
    """
    For each question, checks how many responses already exist in the raw output file,
    and creates tasks only for the missing responses (to reach responses_per_question total).
    Tasks are processed in batches (to respect the rate limit) and new responses are appended
    to the raw output file.
    """
    # Define the raw output file name based on the model name
    output_file = 'output' + model_name.split('/')[-1] + '.jsonl'
    # Load any existing responses
    existing = get_existing_raw_data(output_file)
    
    # Build tasks for missing responses only
    tasks = []
    for q_idx, q_data in enumerate(questions):
        question_text = q_data['question']
        current_indices = existing.get(q_idx, [])
        current_count = len(current_indices)
        missing = responses_per_question - current_count
        if missing > 0:
            for r in range(current_count, responses_per_question):
                tasks.append({
                    "question_id": q_idx,
                    "question": question_text,
                    "response_index": r
                })
    total_tasks = len(tasks)
    print(f"Total new tasks: {total_tasks}")
    
    if total_tasks == 0:
        print("All questions already have the required number of responses.")
        return output_file

    batch_size = 90  # use rate limit of 90 queries per minute
    total_batches = math.ceil(total_tasks / batch_size)
    
    # Open the output file in append mode so that new responses are added
    with open(output_file, "a", encoding="utf-8") as f:
        with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
            task_idx = 0
            for batch in range(total_batches):
                start_batch = time.time()
                current_batch = tasks[task_idx:task_idx+batch_size]
                future_to_task = {}
                # Submit all tasks in the current batch
                for task in current_batch:
                    future = executor.submit(fetch_response, task["question"], model_name)
                    future_to_task[future] = task
                
                # Process completed tasks and write their results
                for future in concurrent.futures.as_completed(future_to_task):
                    task_info = future_to_task[future]
                    try:
                        response_text = future.result()
                    except Exception as e:
                        response_text = f"Error: {e}"
                    output_record = {
                        "question_id": task_info["question_id"],
                        "question": task_info["question"],
                        "response_index": task_info["response_index"],
                        "generated_response": response_text
                    }
                    f.write(json.dumps(output_record) + "\n")
                    f.flush()
                    print(f"Saved response for question {task_info['question_id']} (response index {task_info['response_index']})")
                
                task_idx += batch_size
                # Wait until 60 seconds have passed since the start of the batch to respect rate limits
                elapsed = time.time() - start_batch
                if elapsed < 60:
                    wait_time = 60 - elapsed
                    print(f"Batch {batch + 1}/{total_batches} complete. Waiting {wait_time:.2f} seconds for rate limit.")
                    time.sleep(wait_time)
                else:
                    print(f"Batch {batch + 1}/{total_batches} took {elapsed:.2f} seconds, no wait needed.")
    print(f"All {total_tasks} new responses appended to {output_file}")
    return output_file

def concatenate_responses(input_file, output_file="concatenated_output.jsonl", responses_per_question=64):
    """
    Reads the raw JSONL output file and groups responses by question.
    Each question is assumed to have up to responses_per_question items.
    The final concatenated data is saved to a new JSONL file.
    """
    with open(input_file, "r", encoding="utf-8") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
    
    # Group responses by question_id
    grouped = {}
    for item in raw_data:
        qid = item["question_id"]
        if qid not in grouped:
            grouped[qid] = {
                "question": item["question"],
                "generated_responses": []
            }
        grouped[qid]["generated_responses"].append((item["response_index"], item["generated_response"]))
    
    # Sort responses within each question by response_index and trim to responses_per_question
    concatenated_data = []
    for qid in sorted(grouped.keys()):
        group = grouped[qid]
        sorted_responses = [resp for idx, resp in sorted(group["generated_responses"], key=lambda x: x[0])]
        group["generated_responses"] = sorted_responses[:responses_per_question]
        concatenated_data.append(group)
    
    # Write the concatenated output, one JSON per question
    with open(output_file, "w", encoding="utf-8") as f:
        for item in concatenated_data:
            f.write(json.dumps(item) + "\n")
    print(f"Concatenated output saved to {output_file}")
    print(f"Total questions processed: {len(concatenated_data)}")

if __name__ == "__main__":
    # Read input questions from the JSONL file
    input_file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'
    with open(input_file_path, "r", encoding="utf-8") as f:
        questions = [json.loads(line) for line in f if line.strip()]
    
    responses_per_question = 64  # Total responses required per question
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # Your model of choice
    
    # Generate and append new responses (only for missing ones)
    raw_output_file = get_all_responses(questions, responses_per_question, model_name)
    
    # Concatenate all responses (old + new) per question into the final output file.
    concatenate_responses(raw_output_file, "concatenated_output.jsonl", responses_per_question)


Total new tasks: 805
Saved response for question 0 (response index 61)
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)", "type_": "model_rate_limit"}. Retry 1/3 in 5 seconds...
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increa

In [2]:
cd ..

/Users/sathvikavasarala/Desktop/codes/r1-1.5b-interp


/Users/sathvikavasarala/miniconda3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
import json
import math
import time
import os
import concurrent.futures
from dotenv import load_dotenv
from together import Together

# Load environment variables from .env file and set the API key
load_dotenv()
api_key = os.getenv('TOGETHER_API_KEY')
os.environ["TOGETHER_API_KEY"] = api_key
client = Together()

def fetch_response(question, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", max_retries=30):
    """
    Makes a single API call to fetch a response for the given question using streaming.
    Includes error handling and retries on rate-limit (429) or server overload (503) errors.
    
    Returns:
        str: The concatenated response.
    """
    retry_count = 0
    delay = 5  # initial delay (seconds) before retrying
    while True:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "user",
                        "content": "Please reason step by step, and put your final answer within \\boxed{} " + question
                    },
                ],
                max_tokens=None,
                temperature=0.6,
                top_p=0.95,
                top_k=None,
                repetition_penalty=1,
                stop=["<｜end of sentence｜>"],
                stream=True,
            )
            current_response = []
            for token in response:
                if hasattr(token, 'choices'):
                    current_response.extend([choice.delta.content for choice in token.choices])
            result = "".join(current_response)
        except Exception as e:
            # If an exception is raised, check for rate limit (429) or server overload (503) errors
            if "429" in str(e) and retry_count < max_retries:
                retry_count += 1
                print(f"Exception hit: {e}. Retry {retry_count}/{max_retries} in {delay} seconds...")
                time.sleep(delay)
                delay *= 2
                continue
            raise e

        # Check if the result itself is an error message (even if no exception was thrown)
        if result.strip().lower().startswith("error") and (("429" in result or "503" in result) and retry_count < max_retries):
            retry_count += 1
            print(f"Received error response: {result}. Retrying {retry_count}/{max_retries} in {delay} seconds...")
            time.sleep(delay)
            delay *= 2
            continue

        return result

def get_existing_raw_data(output_file):
    """
    Reads the raw output file (if it exists) and returns a dictionary mapping each question_id
    to the list of response indices already generated, filtering out any responses that start with "error".
    """
    existing = {}
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    record = json.loads(line)
                    # Skip this record if the generated response starts with "error"
                    response = record.get("generated_response", "")
                    if response.strip().lower().startswith("error"):
                        continue
                    qid = record["question_id"]
                    if qid not in existing:
                        existing[qid] = []
                    existing[qid].append(record["response_index"])
    return existing

def get_all_responses(questions, responses_per_question=64, model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"):
    """
    For each question, checks how many responses already exist in the raw output file,
    and creates tasks only for the missing responses (to reach responses_per_question total).
    Tasks are processed in batches (to respect the rate limit) and new responses are appended
    to the raw output file.
    """
    # Define the raw output file name based on the model name
    output_file = 'output' + model_name.split('/')[-1] + '.jsonl'
    # Load any existing responses
    existing = get_existing_raw_data(output_file)
    
    # Build tasks for missing responses only
    tasks = []
    for q_idx, q_data in enumerate(questions):
        question_text = q_data['question']
        current_indices = existing.get(q_idx, [])
        current_count = len(current_indices)
        missing = responses_per_question - current_count
        if missing > 0:
            for r in range(current_count, responses_per_question):
                tasks.append({
                    "question_id": q_idx,
                    "question": question_text,
                    "response_index": r
                })
    total_tasks = len(tasks)
    print(f"Total new tasks: {total_tasks}")
    
    if total_tasks == 0:
        print("All questions already have the required number of responses.")
        return output_file

    batch_size = 90  # use rate limit of 90 queries per minute
    total_batches = math.ceil(total_tasks / batch_size)
    
    # Open the output file in append mode so that new responses are added
    with open(output_file, "a", encoding="utf-8") as f:
        with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
            task_idx = 0
            for batch in range(total_batches):
                start_batch = time.time()
                current_batch = tasks[task_idx:task_idx+batch_size]
                future_to_task = {}
                # Submit all tasks in the current batch
                for task in current_batch:
                    future = executor.submit(fetch_response, task["question"], model_name)
                    future_to_task[future] = task
                
                # Process completed tasks and write their results
                for future in concurrent.futures.as_completed(future_to_task):
                    task_info = future_to_task[future]
                    try:
                        response_text = future.result()
                    except Exception as e:
                        response_text = f"Error: {e}"
                    output_record = {
                        "question_id": task_info["question_id"],
                        "question": task_info["question"],
                        "response_index": task_info["response_index"],
                        "generated_response": response_text
                    }
                    f.write(json.dumps(output_record) + "\n")
                    f.flush()
                    print(f"Saved response for question {task_info['question_id']} (response index {task_info['response_index']})")
                
                task_idx += batch_size
                # Wait until 60 seconds have passed since the start of the batch to respect rate limits
                elapsed = time.time() - start_batch
                if elapsed < 60:
                    wait_time = 60 - elapsed
                    print(f"Batch {batch + 1}/{total_batches} complete. Waiting {wait_time:.2f} seconds for rate limit.")
                    time.sleep(wait_time)
                else:
                    print(f"Batch {batch + 1}/{total_batches} took {elapsed:.2f} seconds, no wait needed.")
    print(f"All {total_tasks} new responses appended to {output_file}")
    return output_file

def concatenate_responses(input_file, output_file="concatenated_output.jsonl", responses_per_question=64):
    """
    Reads the raw JSONL output file and groups responses by question.
    Each question is assumed to have up to responses_per_question items.
    The final concatenated data is saved to a new JSONL file.
    """
    with open(input_file, "r", encoding="utf-8") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
    
    # Group responses by question_id
    grouped = {}
    for item in raw_data:
        qid = item["question_id"]
        if qid not in grouped:
            grouped[qid] = {
                "question": item["question"],
                "generated_responses": []
            }
        grouped[qid]["generated_responses"].append((item["response_index"], item["generated_response"]))
    
    # Sort responses within each question by response_index and trim to responses_per_question
    concatenated_data = []
    for qid in sorted(grouped.keys()):
        group = grouped[qid]
        sorted_responses = [resp for idx, resp in sorted(group["generated_responses"], key=lambda x: x[0])]
        group["generated_responses"] = sorted_responses[:responses_per_question]
        concatenated_data.append(group)
    
    # Write the concatenated output, one JSON per question
    with open(output_file, "w", encoding="utf-8") as f:
        for item in concatenated_data:
            f.write(json.dumps(item) + "\n")
    print(f"Concatenated output saved to {output_file}")
    print(f"Total questions processed: {len(concatenated_data)}")

In [5]:
if __name__ == "__main__":
    # Read input questions from the JSONL file
    input_file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'
    with open(input_file_path, "r", encoding="utf-8") as f:
        questions = [json.loads(line) for line in f if line.strip()]
    
    responses_per_question = 64  # Total responses required per question
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # Your model of choice
    
    # Generate and append new responses (only for missing ones)
    raw_output_file = get_all_responses(questions, responses_per_question, model_name)
    
    # Concatenate all responses (old + new) per question into the final output file.
    concatenate_responses(raw_output_file, "concatenated_output.jsonl", responses_per_question)

Total new tasks: 396
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)", "type_": "model_rate_limit"}. Retry 1/3 in 5 seconds...
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contac

In [7]:
if __name__ == "__main__":
    # Read input questions from the JSONL file
    input_file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'
    with open(input_file_path, "r", encoding="utf-8") as f:
        questions = [json.loads(line) for line in f if line.strip()]
    
    responses_per_question = 64  # Total responses required per question
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # Your model of choice
    
    # Generate and append new responses (only for missing ones)
    raw_output_file = get_all_responses(questions, responses_per_question, model_name)
    
    # Concatenate all responses (old + new) per question into the final output file.
    concatenate_responses(raw_output_file, "concatenated_output.jsonl", responses_per_question)

Total new tasks: 247
Saved response for question 9 (response index 59)
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)", "type_": "model_rate_limit"}. Retry 1/30 in 5 seconds...
Exception hit: Error code: 429 - {"message": "You have reached the rate limit specific to this model deepseek-ai/DeepSeek-R1-Distill-Qwen-14B. The maximum rate limit for this model is 90.0 queries per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about incre

In [8]:
if __name__ == "__main__":
    # Read input questions from the JSONL file
    input_file_path = 'aime-eval-runs/test_qwen-instruct_t0.6_k64_s0_e30_agenticorg_deepscaler.jsonl'
    with open(input_file_path, "r", encoding="utf-8") as f:
        questions = [json.loads(line) for line in f if line.strip()]
    
    responses_per_question = 64  # Total responses required per question
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # Your model of choice
    
    # Generate and append new responses (only for missing ones)
    raw_output_file = get_all_responses(questions, responses_per_question, model_name)
    
    # Concatenate all responses (old + new) per question into the final output file.
    concatenate_responses(raw_output_file, "concatenated_output.jsonl", responses_per_question)

Total new tasks: 16
Saved response for question 9 (response index 63)
Saved response for question 8 (response index 63)
Saved response for question 12 (response index 63)
Saved response for question 5 (response index 62)
Saved response for question 4 (response index 63)
Saved response for question 14 (response index 63)
Saved response for question 5 (response index 60)
Saved response for question 5 (response index 63)
Saved response for question 1 (response index 63)
Saved response for question 6 (response index 63)
Saved response for question 14 (response index 62)
Saved response for question 2 (response index 63)
Saved response for question 3 (response index 63)
Saved response for question 6 (response index 62)
Saved response for question 2 (response index 62)
Saved response for question 5 (response index 61)
Batch 1/1 took 185.56 seconds, no wait needed.
All 16 new responses appended to outputDeepSeek-R1-Distill-Qwen-14B.jsonl
Concatenated output saved to concatenated_output.jsonl
T